In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu, corpus_bleu
from nsd_access import NSDAccess
import os, sys
from tqdm import tqdm
import glob

2022-06-08 11:23:03.451355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/Cluster-Apps/intel/2017.4/compilers_and_libraries_2017.4.196/linux/daal/lib/intel64_lin:/usr/local/Cluster-Apps/intel/2017.4/compilers_and_libraries_2017.4.196/linux/daal/../tbb/lib/intel64_lin/gcc4.4:/usr/local/Cluster-Apps/intel/2017.4/compilers_and_libraries_2017.4.196/linux/ipp/lib/intel64:/usr/local/Cluster-Apps/intel/2017.4/compilers_and_libraries_2017.4.196/linux/tbb/lib/intel64/gcc4.7:/usr/local/Cluster-Apps/intel/2017.4/debugger_2017/iga/lib:/usr/local/Cluster-Apps/intel/2017.4/debugger_2017/libipt/intel64/lib:/usr/local/Cluster-Apps/intel/2017.4/compilers_and_libraries_2017.4.196/linux/mpi/intel64/lib:/usr/local/Cluster-Apps/intel/2017.4/compilers_and_libraries_2017.4.196/linux/tbb/lib/intel64_lin/gcc4.7:/usr/local/Cluste

In [2]:
home_dir = f'/home/hpcgies1/Masters-Thesis/AttemptFour/'
eval_dir = f'{home_dir}/Log/'

# model = 'all_subjects'
# epoch = 71
# model = 'subject_2_baseline2'
# epoch = 80
# model = 'subject_2_both_layer_norm'
# epoch = 25
# model = 'subject_2_lstm_layer_norm'
# epoch = 44
# model = 'subject_2_dot_product'
# epoch = '46'

models = ['subject_1_baseline', 'subject_2_baseline2', 'subject_5_baseline', 'subject_7_baseline']

model = models[0]

model_path   = glob.glob(f"{eval_dir}/{model}/eval_out/output_captions_*")[0]
# model_path = f'/home/hpcgies1/Masters-Thesis/AttemptFour/Log/{model}/eval_out/output_captions_{epoch}.npy'


## Auxiliary functions

In [3]:
nsd_loader = NSDAccess('/home/hpcgies1/rds/hpc-work/NIC/NSD/')
nsd_loader.stim_descriptions = pd.read_csv(nsd_loader.stimuli_description_file, index_col=0)

In [4]:
def load_data(fname):
    return np.squeeze(np.load(open(fname, 'rb')), axis=-1)

def load_tokenizer(fname):
    with open(fname, 'r') as f:
        tok =tokenizer_from_json(f.read())
    return tok

def remove_pad_end(cap: str):
    cap = cap.split(" ")
    cap = [i for i in cap if i != '<pad>' and i != '<end>']
    return " ".join(cap)

def get_target_caption(key):
    """ Return target caption for a given key in [1,73000] """
    with HiddenPrints():
        target = nsd_loader.read_image_coco_info([int(key)-1]) # returns list(dict)
        target = target[0]['caption'] # get first target caption
    return target

def get_target_captions(keys: list):
    """ Return target caption for a given key in [1,73000] """
    keys = [int(i)-1 for i in keys]
    output_targets = []
    with HiddenPrints():
        targets = nsd_loader.read_image_coco_info(keys) # returns list(list(dict))
    for _, t in enumerate(targets):
        ts = []
        for i in range(5):
            target = t[i]['caption'] # get target captions
            ts.append(target)
        output_targets.append(ts)
    return output_targets

def clean_targets(targets: list):
    """ given list of list of targets: return cleaned strings """
    new = []
    for i in range(len(targets)):
        ts = []
        for k in range(5):
            t = targets[i][k]
            t = t.replace("."," ").replace(",", " ").strip().split(" ")
            t = [n.lower() for n in t if n != '']
            t = " ".join(t)
            ts.append(t)
        new.append(ts)
    return new

class HiddenPrints:
    """ Use with with HiddenPrints() to temporarily surpress print output """
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

## Load data

In [58]:
tokenizer = load_tokenizer(f'/home/hpcgies1/Masters-Thesis/AttemptFour/Log/{model}/eval_out/tokenizer.json')
test_keys = pd.read_csv(f'{home_dir}/TrainData/subj02_conditions2.csv')
test_keys = test_keys['nsd_key'].loc[test_keys['is_test'] == 1].values
output = load_data(model_path)
output.shape

(515, 15)

In [14]:
tokenizer = load_tokenizer(f'/home/hpcgies1/rds/hpc-work/NIC/Log/torch_alt_batches/eval_out/tokenizer.json')
test_keys = pd.read_csv(f'{home_dir}/TrainData/subj02_conditions2.csv')
test_keys = test_keys['nsd_key'].loc[test_keys['is_test'] == 1].values
output = np.load('/home/hpcgies1/rds/hpc-work/NIC/Log/torch_alt_batches/eval_out/output_captions_8.npy')
output = output[1]
output.shape

(515, 15)

In [15]:
captions = tokenizer.sequences_to_texts(output)
print(len(captions))
targets = get_target_captions(test_keys)
targets = clean_targets(targets)
print(len(targets))

515
515


In [11]:
def compute_bleu(captions: list, targets: list):
    captions = [remove_pad_end(c) for c in captions]
    
    weights = [
        (1, 0, 0, 0),
        (0, 1, 0, 0),
        (0, 0, 1, 0),
        (0, 0, 0, 1),
        (1./1., 0, 0, 0),
        (1./2., 1./2., 0, 0),
        (1./3., 1./3., 1./3., 0),
        (1./4., 1./4., 1./4., 1./4.)
    ]
    
    hypothesis = []
    references = []
    for i in range(1):
        caps = captions[i*515:i*515+515]
        for i in range(len(caps)):
            ref = [i.split(" ") for i in targets[i]]
            hyp = caps[i].split(" ")
            hypothesis.append(hyp)
            references.append(ref)

    chencherry = SmoothingFunction()
    for w in weights[4:]:
        b_score = corpus_bleu(references, hypothesis, weights=w, smoothing_function=chencherry.method0)
        print(b_score)
    return b_score

bleu = compute_bleu(captions, targets)
# print(bleu)

0.4985661614092585
0.2926175940448853
0.1740020290350698
0.11607832773724937


In [74]:
def bleu_(candidate, targets):
    """ Compute BLEU between a candidate caption and its targets
    Parameters:
    -----------
        candidate : str
        targets : list of strings
    """
    candidate = candidate.split(" ")
    candidate = [remove_pad_end(c) for c in candidate]
    
    weights = [
        (1, 0, 0, 0),  # Bleu-1
        (0, 0, 0, 1),  # Bleu-4
    ]
    
    
    hypothesis = []
    references = []
    for i in range(5):
        ref = targets[i].split(" ")
        references.append(ref)

    
    chencherry = SmoothingFunction()
    b_scores = []
    for w in weights:
        b_score = sentence_bleu(references, candidate, weights=w, smoothing_function=chencherry.method1)
        b_scores.append(b_score)
    return b_scores

In [77]:
for i in range(10):
    print(*bleu_(captions[i], targets[i]))
    print(f"\t{captions[i]}")
    print(f"\t{targets[i][0]}")
    print()

0.3333333333333333 0.008333333333333335
	a man riding a skateboard down a street <end> <pad> <pad> <pad> <pad> <pad> <pad>
	a person in a wetsuit surfing on a turquoise wave

0.4 0.08333333333333333
	a man riding a wave on top of a surfboard <end> <pad> <pad> <pad> <pad>
	a windsurfer some water a hill sand and some kayaks

0.5333333333333333 0.008333333333333335
	a plate of food with a fork and a fork <end> <pad> <pad> <pad> <pad>
	a plate of yummy food of some kind

0.5333333333333333 0.08333333333333333
	a plate with a sandwich and a fork on it <end> <pad> <pad> <pad> <pad>
	a cup of coffee on a plate with a spoon

0.26666666666666666 0.008333333333333335
	a man is standing on a tennis court holding a racquet <end> <pad> <pad> <pad>
	three young women are trying to catch a frisbee

0.4666666666666667 0.008333333333333335
	a large airplane flying through the sky <end> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	a lone airplane is flying high against a grey sky

0.26666666666666666 0.0